In [1]:
import json
from nltk_utils import tokenize, stem,bag_of_words
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model import NeuralNet
from nltk.stem.porter import PorterStemmer
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


How are you'll?
['How', 'are', 'you', "'ll", '?']
['organ', 'organ', 'organ']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open('intents.json','r')as f:
    intents_dict = json.load(f)


In [4]:
all_words = []
tags = []
xy = []

In [5]:
for intent in intents_dict['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag)) 


In [6]:
ignore_verbs = ['?', '!', ',', '.']
all_words = [stem(w) for w in all_words if w not in ignore_verbs]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [7]:
X_train = [] #bag of words
y_train = []
for (pattern_sentence,tag) in xy:
    bag = bag_of_words(pattern_sentence,all_words)
    X_train.append(bag)

    
    label = tags.index(tag)
    y_train.append(label) #CrossEntropyLossX_

X_train = np.array(X_train)
y_train = np.array(y_train)


In [13]:
#hyperparameter
batch_size = 8
hidden_size = 8#no of neurons in the hidden layer
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 2000

In [14]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
        
    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    
    
dataset = ChatDataset()
train_loader = DataLoader(dataset= dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)



In [16]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [17]:
for epoch in range(num_epochs):
    for(words,labels) in train_loader:
        words = words.to(device)
        labels = labels.to(torch.long)
        labels = labels.to(device)
        #forward pass
        outputs = model(words)
        loss = criterion(outputs,labels)
        #backward and optimizer
        optimizer.zero_grad() # emptying the gradient
        loss.backward()#compute gradients by backpropagation
        optimizer.step()#update model parameters using optimizer
    if(epoch + 1) % 100 == 0:
        print(f'epoch{epoch+1}/{num_epochs}, loss = {float(loss.item()):.4f}')
print(f'final loss, loss = {loss.item():.4f}')
        

epoch100/2000, loss = 1.2563
epoch200/2000, loss = 0.3431
epoch300/2000, loss = 0.2949
epoch400/2000, loss = 0.2911
epoch500/2000, loss = 0.0675
epoch600/2000, loss = 0.0025
epoch700/2000, loss = 0.0021
epoch800/2000, loss = 0.0041
epoch900/2000, loss = 0.0007
epoch1000/2000, loss = 0.0030
epoch1100/2000, loss = 0.0015
epoch1200/2000, loss = 0.0007
epoch1300/2000, loss = 0.0005
epoch1400/2000, loss = 0.0007
epoch1500/2000, loss = 0.0004
epoch1600/2000, loss = 0.0003
epoch1700/2000, loss = 0.0001
epoch1800/2000, loss = 0.0003
epoch1900/2000, loss = 0.0001
epoch2000/2000, loss = 0.0001
final loss, loss = 0.0001


In [22]:
data ={
    "model_state":model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
    
}

FILE = "data.pth"
torch.save(data,FILE)

print(f'training complete.file saved to {FILE}')

training complete.file saved to data.pth
